In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

import torch.nn.functional as F
import numpy as np
import torch
from torch import nn
from torch.nn import init

In [2]:
class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1,reduction=16):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        
        self.relu = nn.ReLU(inplace=True)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [3]:
def resnet34(num_classes, in_channels=1):
    
    def resnet_block(in_channels, out_channels, num_residuals,
                     first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual(in_channels, out_channels,
                                        use_1x1conv=True, strides=2))
            else:
                blk.append(Residual(out_channels, out_channels))
        return nn.Sequential(*blk)

    
    net = nn.Sequential(
        nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU())
    net.add_module("resnet_block1", resnet_block(64, 64, 3, first_block=True))
    net.add_module("resnet_block2", resnet_block(64, 128, 4))
    net.add_module("resnet_block3", resnet_block(128, 256, 6))
    net.add_module("resnet_block4", resnet_block(256, 512, 3))
    net.add_module("global_avg_pool", nn.AdaptiveAvgPool2d((1,1)))
    net.add_module("fc", nn.Sequential(nn.Flatten(),
                                       nn.Linear(512, num_classes)))
    return net

In [4]:
def get_net():
    num_classes = 5
    net = resnet34(num_classes, 3)
    return net

model=get_net()

In [5]:
model.load_state_dict(torch.load('./aaa/aaa/re34normal/network_params.pth'))
model=model.eval()

In [6]:
datadir="/home/omnisky/shenxiaochen/"
transform_test = torchvision.transforms.Compose([
    

    torchvision.transforms.CenterCrop(96),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

test_ds= torchvision.datasets.ImageFolder(
    os.path.join(datadir, 'test'),
    transform=transform_test)

batch_size=128
test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

def evaluate_accuracy_gpu(net, data_iter, device=None): #@save
  
    if isinstance(net, torch.nn.Module):
        net.eval()  
        if not device:
            device = next(iter(net.parameters())).device
    
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            
            X = [x.to(device) for x in X]
        else:
            X = X.to(device)
        y = y.to(device)
        metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [7]:
# accuracy valid
device=d2l.try_gpu()
evaluate_accuracy_gpu(model.to(device),test_iter)

0.9405446873801304

In [8]:
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score,precision_score,recall_score,f1_score

In [9]:
# confusion matrix
model.to(device)
model.eval()
conf_mat=np.zeros((5,5))
for i,data in enumerate(test_iter):
    inputs, labels=data
    inputs, labels= inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs=model(inputs)
        _, predicted = torch.max(outputs.data,1)
        for j in range(len(labels)):
            cate_i=labels[j].cpu().numpy()
            pre_i =predicted[j].cpu().numpy()
            conf_mat[cate_i,pre_i]+=1

In [11]:
valid_confusematrix=pd.DataFrame(conf_mat)

In [12]:
valid_confusematrix

,0,1,2,3,4
0,1060.0,3.0,12.0,0.0,1.0
1,1.0,1139.0,1.0,0.0,7.0
2,36.0,1.0,918.0,6.0,1.0
3,103.0,0.0,138.0,769.0,0.0
4,0.0,0.0,0.0,0.0,1018.0


In [13]:
preds=[]
true=[]
with torch.no_grad():
    for x,y in test_iter:
        x=x.to(device)
        y=y.to(device)
        output=model(x)
        output=output.argmax(dim=1)
        preds.extend(output.detach().cpu().numpy())
        true.extend(y.detach().cpu().numpy())
        
    

In [14]:
accuracy_score(true,preds)

0.9405446873801304

In [15]:
recall_score(true,preds,average='macro')

0.9385876966291248

In [16]:
f1_score(true,preds,average='macro')

0.9373992352995449

In [17]:
precision_score(true,preds,average='macro')

0.9444149877666403

In [16]:
from sklearn.preprocessing import label_binarize

In [17]:
test_trues=label_binarize(true,classes=[i for i in range(5)])
test_preds=label_binarize(preds,classes=[i for i in range(5)])

In [18]:
roc_auc_score(test_trues.ravel(),test_preds.ravel())# micro

0.9628404296125815

In [18]:
outsidetest_ds= torchvision.datasets.ImageFolder(
    os.path.join(datadir, 'outside_test'),
    transform=transform_test) 
outsidetest_iter = torch.utils.data.DataLoader(outsidetest_ds, batch_size, shuffle=False,
                                        drop_last=False)

In [19]:
evaluate_accuracy_gpu(model.to(device),outsidetest_iter)

0.8122725618631732

In [20]:
# confusion matrix
model.to(device)
model.eval()
conf_mat=np.zeros((5,5))
for i,data in enumerate(outsidetest_iter):
    inputs, labels=data
    inputs, labels= inputs.to(device), labels.to(device)
    with torch.no_grad():
        outputs=model(inputs)
        _, predicted = torch.max(outputs.data,1)
        for j in range(len(labels)):
            cate_i=labels[j].cpu().numpy()
            pre_i =predicted[j].cpu().numpy()
            conf_mat[cate_i,pre_i]+=1

In [21]:
pd.DataFrame(conf_mat)

,0,1,2,3,4
0,5153.0,1.0,121.0,1.0,13.0
1,163.0,4258.0,200.0,0.0,606.0
2,201.0,1.0,4367.0,454.0,1.0
3,479.0,2.0,1641.0,2993.0,0.0
4,218.0,12.0,13.0,0.0,1086.0


In [21]:
o_preds=[]
o_true=[]
with torch.no_grad():
    for x,y in outsidetest_iter:
        x=x.to(device)
        y=y.to(device)
        output=model(x)
        output=output.argmax(dim=1)
        o_preds.extend(output.detach().cpu().numpy())
        o_true.extend(y.detach().cpu().numpy())

In [22]:
accuracy_score(o_true,o_preds)

0.8122725618631732

In [23]:
recall_score(o_true,o_preds,average='macro')

0.8120855744908824

In [24]:
f1_score(o_true,o_preds,average='macro')

0.7950807738527691

In [25]:
precision_score(o_true,o_preds,average='macro')

0.8037426451293062

In [26]:
test_trues=label_binarize(o_true,classes=[i for i in range(5)])
test_preds=label_binarize(o_preds,classes=[i for i in range(5)])

In [27]:
roc_auc_score(test_trues.ravel(),test_preds.ravel())

0.8826703511644832